In [22]:
import time
import pandas as pd
import sys
import os
import csv
import imp
import operator

import numpy as np
import nltk

from sklearn import linear_model, metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:

# train model using training set
# test model using test set output results to file
input_cols = ['X','y']

def load_file(file_name):
    return pd.read_csv(file_name, sep=',')

training_set = load_file('TraningDatav1.csv')

In [24]:
training_set['Pro']

0           Pro Gov
1           Pro Gov
2           Pro Gov
3               NaN
4           Pro Gov
5           Pro Gov
6               NaN
7           Pro Gov
8               NaN
9               NaN
10              NaN
11              NaN
12              NaN
13          Pro Gov
14          Pro Gov
15              NaN
16              NaN
17              NaN
18              NaN
19              NaN
20              NaN
21              NaN
22              NaN
23              NaN
24              NaN
25              NaN
26              NaN
27              NaN
28              NaN
29              NaN
           ...     
997             NaN
998             NaN
999     Against Gov
1000            NaN
1001    Against Gov
1002    Against Gov
1003            NaN
1004            NaN
1005            NaN
1006    Against Gov
1007    Against Gov
1008    Against Gov
1009            NaN
1010    Against Gov
1011            NaN
1012    Against Gov
1013    Against Gov
1014            NaN
1015    Against Gov


In [25]:
training_set.dropna(axis=0, how='any')

,Date,Opinion,Pro,TweetText
0,3/16/19 3:30,Positive,Pro Gov,Your Chowkidar is standing firm &amp; serving ...
1,3/15/19 13:02,Positive,Pro Gov,"This is a New India, where a person\xe2\x80\x9..."
2,3/15/19 12:57,Positive,Pro Gov,Public transport is convenient and comfortable...
4,3/15/19 12:50,Positive,Pro Gov,Honoured that we have got the opportunity to w...
5,3/15/19 12:49,Positive,Pro Gov,Thanks for the faith in our work. https://t.co...
7,3/14/19 13:23,Positive,Pro Gov,Joined the ceremony to mark the presentation o...
13,3/14/19 11:24,Neutral,Pro Gov,"With the blessings of 130 crore Indians, what ..."
14,3/13/19 12:40,Neutral,Pro Gov,No place for middlemen.\n\nNo tolerance for co...
57,3/10/19 12:43,Positive,Pro Gov,"Guided by \xe2\x80\x98Sabka Saath, Sabka Vikas..."
58,3/10/19 12:41,Positive,Pro Gov,India is proud that:\n\n2.5 crore families hav...


In [26]:
# helper functions:
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    # Use lemmatize
    #stems = lemmatize_tokens(tokens, lemmatizer)
    # Use stemming
    stems = stem_tokens(tokens, stemmer)
    return stems

stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens, lemmatizer):
    lemmatized = []
    for item in tokens:
        lemmatized.append( lemmatizer.lemmatize( item ) )
    return lemmatized

In [27]:
# train model:
def train(train_text, train_labels):
    global bow_transformer, tfidf_transformer 
    
    
    ## Choose your model!
    #model = linear_model.LogisticRegression(C=5, class_weight='balanced', fit_intercept=0, multi_class='ovr', solver='newton-cg')
    model = linear_model.LogisticRegression(C=10, class_weight='balanced', fit_intercept=0, multi_class='ovr', solver='liblinear')
    
    #model = SVC(C=0.9, kernel='linear', probability=True)
    
#     model = LinearSVC(C=1, class_weight= 'balanced', dual=True, fit_intercept=0, loss= 'hinge', multi_class='ovr',
#                       penalty='l2')
    
    X_train = train_text
    y_train = train_labels
   
    bow_transformer = CountVectorizer(ngram_range=(
    1, 3), analyzer='word', stop_words='english', tokenizer= tokenize).fit(X_train)

    X_train_dtm = bow_transformer.transform(X_train)     
    tfidf_transformer = TfidfTransformer().fit(X_train_dtm)
    X_train_dtm = tfidf_transformer.transform(X_train_dtm)

    return model.fit(X_train_dtm, y_train)

In [28]:
trained_model = train(training_set['TweetText'], training_set['Opinion'])

In [29]:
X_test = load_file('UntrainedDatav1.csv')
X_test_dtm = bow_transformer.transform(X_test['TweetText'])
X_test_dtm = tfidf_transformer.transform(X_test_dtm)
y_predict_class = trained_model.predict(X_test_dtm)

print(y_predict_class)
test_results = pd.concat([X_test,pd.Series(y_predict_class)],axis=1,names=['TweetText','Name','Opinion'])
print(test_results)
test_results.columns=['TweetText','Name','Opinion']
test_results.to_csv('TrainedData2v2.csv',index=False)

['Positive' 'Positive' 'Positive' ... 'Negative' 'Negative' 'Positive']
                                              TweetText      Name         0
0     b'Had a wonderful interaction with recipients ...      Namo  Positive
1     b'Today\xe2\x80\x99s India will not be helples...      Namo  Positive
2     b'Looking forward to interacting with young in...      Namo  Positive
3     b'Handed over a \xe2\x80\x9cChadar\xe2\x80\x9d...      Namo  Positive
4     b'Addressing the Construction Technology India...      Namo  Positive
5     b'Welcome Home Wing Commander Abhinandan!\n\nT...      Namo  Positive
6     b'Thank you Andhra Pradesh for the phenomenal ...      Namo  Positive
7     b'Addressing a rally in Kanyakumari, Tamil Nad...      Namo  Positive
8     b'Greetings to my friend and Bihar\xe2\x80\x99...      Namo  Positive
9     b'Interacting with crores of BJP Karyakartas a...      Namo  Positive
10    b'Speaking at the National Youth Parliament Fe...      Namo  Positive
11    b'Travelle